# Librerias

In [13]:
import importlib
import os, shutil
import numpy as np
import pandas as pd
import mplcursors as mpc

from tkinter import *
from tkinter.messagebox import askokcancel

from IPython.display import clear_output

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import plotly.io as pio
pio.renderers.default = 'browser'

import matplotlib.pyplot as plt   # plotting


import rai_exp_2023_sivarita as exp
importlib.reload(exp)

import Sivarita
importlib.reload(Sivarita)

<module 'Sivarita' from 'c:\\repositorios\\rai_exp_2023_sivarita\\Sivarita.py'>

# Preparar datos

Sincronizar los datos de la carpeta <font color='orange'>**/\_data\_**</font> con la carpeta <font color='orange'>**/Experimentaciones/2023_sivarita/\_data\_**</font> de teams.

<font color='red'>**[IMPORTANTE]**</font> Es necesario tener sincronizado el canal [Experimentaciones](https://miumh.sharepoint.com/:f:/s/NBIO/ErXH5_wtbyZBpVINdmT_GRUBNEXJmMrEtJXeEJYZERJYrA?e=WnxzMY) de Teams con el ordenador.

In [ ]:
if not "dataSyncCheck" in locals():
    exp.downloadDataFromTeams("/Resultados/_data_")

dataSyncCheck = True

Cargar excel con los usuarios

In [2]:
path = "./_data_/"

df_brazo = pd.read_excel(path + "participantes_datos_analisis.xlsx")
df_brazo.sample(9)

,nombre,ID,brazo,FM,grupo
18,Juan Manuel_Sanchez Plaza,20,L,66,leve
20,Fernando_Ruiz Santa Olalla,22,R,23,moderado
0,Rosa_Ivorra,1,R,66,leve
4,Isabel_Cuenca,5,R,66,leve
23,Vicente_Estrela,25,R,66,leve
33,Jose Tomas_Torralbo,35,L,42,moderado
10,Teresa_García,11,R,64,leve
15,Angela_Mayor,16,R,64,leve
14,Jose Andres_Ferrer Ferrer,15,R,42,moderado


Cargar datos usuarios

In [3]:
#Path
path = "./_data_/Usuarios/"

#Cargar datos
df_data = exp.loadData(path)
df_data.sample(5)

,usuario,sesion,actividad,tipo_actividad,upper_size,fore_size,brazo,num_rep,dataFrame,dataFrameIA
398,Raúl_Martín,2023_10_10_12_43_27,Move_Cube,DESPLAZAR,325.0,280.0,R,4.0,q1 q2 q3 q4...,tarea data_org_1 data_pred_1 data_org_2 ...
98,Carmen _Saez,2023_3_23_12_3_6,Move_Cube,DEPOSITAR,290.0,260.0,R,4.0,q1 q2 q3 q...,tarea data_org_1 data_pred_1 data_org_2 ...
15,Alberto_Villegas,2023_3_27_12_48_45,Move_Cube,DESPLAZAR,290.0,280.0,R,4.0,q1 q2 q3 ...,tarea data_org_1 data_pred_1 data_org_2 ...
8,Abigail_Jaen Llorens,2023_8_14_11_17_48,TouchGame,HOMBRO,310.0,270.0,R,3.0,q1 q2 q3 ...,tarea data_org_1 data_pred_1 data_org_2 ...
443,Vicenta_Beneyto,2023_3_23_10_10_35,TouchGame,NARIZ,300.0,250.0,R,4.0,q1 q2 q3 ...,tarea data_org_1 data_pred_1 data_org_2 ...


Filtrar datos erróneos

In [4]:
count_vacio = 0
count_correcto = 0

#Filtrar DataFrames IA vacios
for i_row in range(df_data.shape[0]):
    if df_data.dataFrameIA[i_row].empty is True:
        print("Usuario: {0} -- Sesion: {1}".format(df_data.usuario[i_row], (df_data.sesion[i_row])))
        df_data = df_data.drop(i_row)
        count_vacio = count_vacio + 1
    else:
        count_correcto = count_correcto + 1

df_data.reset_index(drop=True, inplace=True)
print("Correctos: {0}".format(count_correcto))
print("Vacíos: {0}".format(count_vacio))

#Filtrar Dimensiones de brazo fuera de rango
for i_row in range(df_data.shape[0]):
    if (df_data.upper_size[i_row] + df_data.fore_size[i_row]) < 450 or (df_data.upper_size[i_row] + df_data.fore_size[i_row]) > 700:
        df_data = df_data.drop(i_row)

df_data.reset_index(drop=True, inplace=True)

df_data.sample(5)



Correctos: 466
Vacíos: 0


,usuario,sesion,actividad,tipo_actividad,upper_size,fore_size,brazo,num_rep,dataFrame,dataFrameIA
165,Esther_Blas Cruz,2023_6_12_11_40_43,Move_Cube,DEPOSITAR,280.0,220.0,L,3.0,q1 q2 q3 q...,tarea data_org_1 data_pred_1 data_org_2 ...
44,Angela_Mayor,2023_5_5_9_52_38,MoveGlass,LLENADO,310.0,250.0,R,4.0,q1 q2 q3 q...,tarea data_org_1 data_pred_1 data_org_2 ...
357,María _Martinez,2023_3_27_16_11_8,MoveGlass,LLENADO,310.0,250.0,R,4.0,q1 q2 q3 q...,tarea data_org_1 data_pred_1 data_org_2 ...
218,Isabel_Cuenca,2023_3_23_10_32_44,MoveGlass,LLENADO,280.0,240.0,R,4.0,q1 q2 q3 ...,tarea data_org_1 data_pred_1 data_org_2 ...
369,Paula_Altamirano Deusa,2023_6_12_12_12_29,Move_Cube,SORTEAR,290.0,240.0,L,3.0,q1 q2 q3 ...,tarea data_org_1 data_pred_1 data_org_2 ...


# Procesar Datos

In [5]:
# Obtener los rangos articulares para cada paciente
df_params = exp.process_all_params(df_data)

In [6]:
#Filtrar movimientos articulares
condicion = (abs(df_params.iloc[:,-14:]) > 180).any(axis=1)
df_params = df_params[~condicion]
df_params.sample(5)

,usuario,session,activity,tipo_actividad,brazo,DTW_org_error,DTW_error,Porcentaje_pred,MAX_Q1,MAX_Q2,...,MAX_Q5,MAX_Q6,MAX_Q7,MIN_Q1,MIN_Q2,MIN_Q3,MIN_Q4,MIN_Q5,MIN_Q6,MIN_Q7
350,María Francisca_Perez Mostazo,2023_6_29_10_45_53,PaintForms,TRIÁNGULO,R,12.634668,6.678725,0.000000,54.900543,60.558655,...,96.658301,-13.155964,34.948276,33.354760,36.383186,-33.125259,-65.689361,81.854113,-33.563213,14.308526
268,José Vicente_García,2023_10_10_13_29_14,PaintForms,CUADRADO,R,8.355548,5.199270,0.000000,24.484636,62.855435,...,67.402557,-7.166486,8.061049,-12.409214,20.030600,-74.972769,-63.202559,27.124780,-19.078043,-12.548196
37,Andrea_Blanco,2023_10_10_8_58_59,PaintForms,CUADRADO,R,11.516095,4.012277,0.000000,38.154111,61.589247,...,37.695683,-1.692817,-4.447425,-21.670918,33.665845,-52.610209,-68.607242,19.527416,-9.219353,-19.757380
141,Elías_Belmonte,2023_10_6_13_58_50,Move_Cube,DEPOSITAR,R,3.308802,3.308802,100.000000,41.098321,61.405467,...,92.284366,-16.858215,34.614849,18.591407,33.103460,-34.059799,-41.914376,54.297935,-27.645664,-3.018040
244,Jose Andres_Ferrer Ferrer,2023_4_24_11_27_8,TouchGame,HOMBRO,R,4.996133,4.996133,47.916667,35.890476,62.840019,...,103.894198,14.183467,24.000201,-17.596585,10.819285,-112.370902,-30.621006,41.468017,4.311716,5.689570


In [7]:
#Filtrar por cantidad de actividades
conteo_nombres = df_params['usuario'].value_counts()
print(conteo_nombres)
# Filtra los usuarios que se repiten más de una vez
usuarios_a_mantener = conteo_nombres[conteo_nombres > 1].index
print(len(usuarios_a_mantener))
# Crea un nuevo DataFrame con solo los usuarios que se repiten más de una vez
df_params = df_params[df_params['usuario'].isin(usuarios_a_mantener)]

df_params.sample(5)


Jose Andres_Ferrer Ferrer           21
Juan Manuel_Sanchez Plaza           12
Luis Daniel_Lledó                   12
Jose Tomas_Torralbo                 12
Enric_Agut Pastor                   12
Elías_Belmonte                      12
Cris_García                         12
Clemente_Fernandez                  12
Isabel_Cuenca                       12
Antonio_Montes                      12
Lusia Maria_Pereiro Sousa           12
Raúl_Martín                         12
Angela_Mayor                        12
Francisco_Ballester Marin           11
José Vicente_García                 11
Rosa_Ivorra                         11
Vicenta_Beneyto                     11
Abigail_Jaen Llorens                11
Victoria_Payá                       11
Carmen_Cerrillo                     11
Carmen _Saez                        11
Andrea_Blanco                       11
Ana_Adsuar                          10
Angeles_Cita Garrido                 9
Esther_Blas Cruz                     9
Juan David_Romero        

,usuario,session,activity,tipo_actividad,brazo,DTW_org_error,DTW_error,Porcentaje_pred,MAX_Q1,MAX_Q2,...,MAX_Q5,MAX_Q6,MAX_Q7,MIN_Q1,MIN_Q2,MIN_Q3,MIN_Q4,MIN_Q5,MIN_Q6,MIN_Q7
320,Lusia Maria_Pereiro Sousa,2023_3_23_12_44_40,MoveGlass,LLENADO,L,10.134497,8.951521,0.000000,45.880755,32.129310,...,94.217177,25.254251,0.309412,25.584924,12.434901,-28.825739,-8.344575,53.384802,14.415105,-26.186718
73,Antonio_Montes,2023_7_10_10_32_16,MoveGlass,BEBER,R,12.236382,12.236382,100.000000,51.068761,47.253959,...,98.082741,-13.646399,20.185400,14.665906,19.503362,-29.979674,-24.898203,30.186707,-32.037793,-13.152458
419,Vicente_Estrela,2023_6_14_10_52_3,MoveGlass,LLENADO,L,5.405125,5.405125,100.000000,32.502528,46.349705,...,99.155648,13.111265,33.863844,24.165564,18.717623,-21.510655,-40.341776,31.093612,-17.381708,9.797146
215,Francisco_Mico Porta,2023_6_30_9_44_39,TouchGame,RODILLA,R,8.147837,8.147837,11.111111,20.473251,43.995651,...,55.410526,29.620732,34.431905,3.919439,30.379734,-32.939820,-75.945956,-30.408944,3.478882,-2.973019
159,Enric_Agut Pastor,2023_6_9_12_1_22,TouchGame,HOMBRO,R,7.711051,7.711051,6.250000,6.473488,41.354062,...,85.918735,28.335263,14.936211,-7.438154,2.144236,-79.215045,-28.890961,20.632160,1.337658,-24.600600


Filtrar por Usuario

In [36]:
df_user = df_params[df_params['usuario'] == 'Maria Soledad_Miranda Rosado']
print(df_user)


Empty DataFrame
Columns: [usuario, session, activity, tipo_actividad, brazo, DTW_org_error, DTW_error, Porcentaje_pred, MAX_Q1, MAX_Q2, MAX_Q3, MAX_Q4, MAX_Q5, MAX_Q6, MAX_Q7, MIN_Q1, MIN_Q2, MIN_Q3, MIN_Q4, MIN_Q5, MIN_Q6, MIN_Q7]
Index: []

[0 rows x 22 columns]


Filtrar por Grupos

In [20]:
# for itere grupos
hawer = df_brazo.grupo.unique()
df_group = []
group_list = []
for group in hawer:
    # filtrar grupo
    group_list.append(group)
    df_group.append(exp.process_byGroups(df_brazo, df_params, group))

data = {
    'grupo': group_list,
    'dataframe': df_group
}

df = pd.DataFrame(data)
df.sample(3)






,grupo,dataframe
2,sano,user mean_MAXQ1 mean_MAXQ2...
1,moderado,user mean_MAXQ1 mean...
0,leve,user mean_MAX...


In [21]:
df.dataframe[0]

,user,mean_MAXQ1,mean_MAXQ2,mean_MAXQ3,mean_MAXQ4,mean_MAXQ5,mean_MAXQ6,mean_MAXQ7,mean_MINQ1,mean_MINQ2,mean_MINQ3,mean_MINQ4,mean_MINQ5,mean_MINQ6,mean_MINQ7,mean_dtw_org_error,mean_dtw_error,mean_porc_pred
0,Rosa_Ivorra,67.737296,70.445953,24.268038,-0.875808,111.766492,2.364154,4.312057,8.768310,32.771093,-53.478265,-41.651356,53.002926,-9.823304,-28.112973,8.737805,8.328444,59.406566
1,Vicenta_Beneyto,54.685663,63.218556,10.506325,5.712410,120.338591,2.924594,30.754095,10.178587,26.633507,-52.783396,-34.187326,47.109919,-24.049740,-17.982268,9.167614,8.249105,53.787879
2,Ana_Adsuar,64.882265,61.845839,-3.123410,28.937725,125.257989,-23.748067,55.839072,14.549600,28.319008,-80.867154,-13.023976,61.654379,-49.483771,-0.676803,9.781738,10.816984,53.125000
3,Carmen _Saez,61.793685,62.243996,3.182000,11.636533,119.863865,-6.463219,23.043025,12.906021,35.790054,-58.726365,-23.523440,61.001811,-26.566504,-6.070048,10.007052,8.373453,48.863636
4,Isabel_Cuenca,65.179153,54.044575,5.552751,8.127076,106.339022,4.469273,20.759197,17.160270,16.409510,-51.034135,-36.145300,50.897063,-26.151699,-8.037840,9.512012,9.169067,43.923611
5,Carmen_Cerrillo,65.560624,65.913106,3.261669,17.955885,97.233346,-13.623110,18.907446,8.234789,19.325131,-61.857067,-25.974062,35.840351,-25.760866,-8.805520,7.936286,9.657415,45.833333
6,Angeles_Cita Garrido,57.091367,69.637942,18.754141,-14.239859,103.824947,5.619019,13.135988,-8.775160,40.394355,-65.272969,-44.891867,44.566014,-9.654683,-16.271253,7.445249,8.395303,51.003086
7,Alberto_Villegas,50.620969,65.729528,1.357070,1.303115,103.377320,5.053090,21.430301,-20.710311,29.467253,-80.464749,-51.577719,35.224882,-19.641463,-22.352896,5.261494,6.140372,72.395833
8,Teresa_García,38.746245,55.651438,1.371858,20.256286,108.781928,7.547267,21.158470,2.116262,13.388513,-46.224446,-30.570058,41.620011,-17.215164,-17.547466,7.295186,8.027752,79.861111
9,Filomena_Almansa Consuegra,19.326692,52.616426,-25.575374,9.577544,117.820473,-5.819582,62.946339,-8.617811,19.044604,-67.951265,-30.754649,9.630757,-41.751799,-20.833878,6.148805,6.403700,70.535714


# Gráfico DTW

In [ ]:
# Plot DTW
index = 195
df = df_data.dataFrameIA[index]
tarea = df_data.tipo_actividad[index]
Sivarita.plotDTWparam(df, tarea)


In [ ]:
mean_org_dtw = []
mean_pred_dtw = []
user_list = []
df_DTW_list = []
tarea_pred = []
idAct = []

for index, rowData in df_data.iterrows():
    idAct.append(rowData['tipo_actividad'])
    user_list.append(rowData['usuario'])
    for subIndex, rowIA in rowData['dataFrameIA'].iterrows():
        # Calcular las medias de todas las joints
        sum_org = rowIA['data_org_1'] + rowIA['data_org_2'] + rowIA['data_org_3'] + rowIA['data_org_4'] + rowIA['data_org_5'] + rowIA['data_org_6'] + rowIA['data_org_7']
        sum_pred = rowIA['data_pred_1'] + rowIA['data_pred_2'] + rowIA['data_pred_3'] + rowIA['data_pred_4'] + rowIA['data_pred_5'] + rowIA['data_pred_6'] + rowIA['data_pred_7']
        mean_org = sum_org / 7
        mean_pred = sum_pred / 7

        mean_org_dtw.append(mean_org)
        mean_pred_dtw.append(mean_pred)
        tarea_pred.append(Sivarita.getActivityType2(rowIA['tarea']))

    DTW_mean = {
        'mean_org_dtw': mean_org_dtw,
        'mean_pred_dtw': mean_pred_dtw,
        'tarea_pred': tarea_pred
    }
    df_DTW_mean = pd.DataFrame(DTW_mean)
    df_DTW_list.append(df_DTW_mean)

    mean_org_dtw.clear()
    mean_pred_dtw.clear()
    tarea_pred.clear()
    


DTW = {
    'usuario': user_list,
    'tipo_actividad': idAct,
    'df_DTW_mean': df_DTW_list
}

df_DTW = pd.DataFrame(DTW)



In [ ]:
def processByGroupDTW(df_main, df_DTW, grp):
    group_DTW_list = []
    nombres = df_main[df_main.grupo == grp].nombre.to_list()
    for user in nombres:
        for i, rowDTW in df_DTW.iterrows():
            if(user == rowDTW['usuario']):
                group_DTW_list.append(rowDTW)
    
    group_DTW = pd.DataFrame(group_DTW_list)

    return group_DTW


df_subGroupDTW = []
group_list2 = []
for group in hawer:
    # filtrar grupo
    group_list2.append(group)
    df_subGroupDTW.append(processByGroupDTW(df_brazo, df_DTW, group))


dataDTW = {
'grupo': group_list2,
'dataframeDTW': df_subGroupDTW
}

df_groupDTW = pd.DataFrame(dataDTW)

In [ ]:
aux_df = df_groupDTW.dataframeDTW[2]

Plot DTW

In [ ]:

x_values  = []
y_values = []
for index1, row_group in df_groupDTW.iterrows():
    for index2, sub_row_group in row_group['dataframeDTW'].iterrows():
        if(row_group['grupo'] == 'sano'):
            color_marker = 'red'
        elif(row_group['grupo'] == 'leve'):
            color_marker = 'green'
        elif(row_group['grupo'] == 'moderado'):
            color_marker = 'blue'
        
        for index3, sub_row_DTW in sub_row_group['df_DTW_mean'].iterrows():
            # x_values.append(index2)
            # y_values.append(sub_row_DTW['mean_org_dtw'])
            if((sub_row_DTW['mean_org_dtw'] < 40) or (sub_row_DTW['mean_pred_dtw'] < 40)):
                if( sub_row_DTW['tarea_pred']== sub_row_group['tipo_actividad']):
                    plt.scatter(sub_row_DTW['mean_org_dtw'], sub_row_DTW['mean_pred_dtw'], c=color_marker, marker='o')
                elif( sub_row_DTW['tarea_pred']!= sub_row_group['tipo_actividad']):
                    plt.scatter(sub_row_DTW['mean_org_dtw'], sub_row_DTW['mean_pred_dtw'], c=color_marker, marker='x')


# fig = px.scatter(x=x_values, y=y_values)
# fig.show()
#plt.scatter(x, y)
plt.xlabel('DTW distance to True Gesture')
plt.ylabel('DTW distance to Predicted Gesture')
plt.title('Gráfico de DTW ')
plt.legend()
plt.show()

# Gráficas

Boxplots por Q

In [25]:
q_data = []
q_groups = []

for i_q in range(1,8):

    qg = []
    ql = []
        
    for i_group in [1, 0, 2]:
        
        df_tmp = df.dataframe[i_group]
        
        qg.append(df.grupo[i_group])
        ql.append( ((df_tmp[str("mean_MAXQ"+str(i_q))]) - (df_tmp[str("mean_MINQ"+str(i_q))])).to_list())
        #ql.append( (df_tmp[str("ROM_Q"+str(i_q))]).to_list() )

    q_groups.append(qg)
    q_data.append(ql)

row = [1,1,1,1,2,2,2]
col = [1,2,3,4,1,2,3]

fig = make_subplots(rows=2, cols=4, subplot_titles=("Q1","Q2","Q3","Q4","Q5","Q6","Q7"))
for i_plot in range(7):
    fig.add_trace(go.Box(y=np.array(q_data[i_plot][1]),marker_color=px.colors.qualitative.T10[1],name=q_groups[i_plot][1]),row=row[i_plot],col=col[i_plot])
    fig.add_trace(go.Box(y=np.array(q_data[i_plot][0]),marker_color=px.colors.qualitative.T10[2],name=q_groups[i_plot][0]),row=row[i_plot],col=col[i_plot])
    fig.add_trace(go.Box(y=np.array(q_data[i_plot][2]),marker_color=px.colors.qualitative.T10[3],name=q_groups[i_plot][2]),row=row[i_plot],col=col[i_plot])

fig.update_layout(
    yaxis1 = dict(range=[0, 100]),
    yaxis2 = dict(range=[0, 100]),
    yaxis3 = dict(range=[0, 100]),
    yaxis4 = dict(range=[0, 100]),
    yaxis5 = dict(range=[0, 100]),
    yaxis6 = dict(range=[0, 100]),
    yaxis7 = dict(range=[0, 100]),
    showlegend = False
    )

fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')

fig.show() 

In [54]:
q_data = []
q_groups = []

for i_q in range(1,8):

    qg = []
    ql = []
        
    for i_group in [1, 0, 2]:
        
        df_tmp = df.dataframe[i_group]
        
        qg.append(df.grupo[i_group])
        ql.append( (df_tmp[str("mean_MAXQ"+str(i_q))] - df_tmp[str("mean_MINQ"+str(i_q))]).to_list() )

    q_groups.append(qg)
    q_data.append(ql)

row = [1,1,1,1,2,2,2]
col = [1,2,3,4,1,2,3]

fig = make_subplots(rows=1, cols=1)
for i_plot in range(7):
    
    fig.add_trace(go.Bar(name = q_groups[i_plot][1],
        x=[1+i_plot+0.19],
        y=[np.mean(q_data[i_plot][1])],
        text=[np.round(np.mean(q_data[i_plot][1]),2)],
        textposition="auto",
        textfont_color = "white",
        marker_color=px.colors.qualitative.T10[1],       
        error_y=dict(
            type='data',
            symmetric=True,
            thickness=2,
            width=7,
            array=[np.std(q_data[i_plot][1])]),
        )        
    )
    
    fig.add_trace(go.Bar(name = q_groups[i_plot][0],
        x=[1+i_plot-0.19],
        y=[np.mean(q_data[i_plot][0])],
        text=[np.round(np.mean(q_data[i_plot][0]),2)],
        textposition="auto",
        textfont_color = "white",
        marker_color=px.colors.qualitative.T10[0],            
        error_y=dict(
            type='data',
            symmetric=True,
            thickness=2,
            width=7,
            array=[np.std(q_data[i_plot][0])]),
        )
    )

    fig.add_trace(go.Bar(name = q_groups[i_plot][2],
        x=[1+i_plot-0.19],
        y=[np.mean(q_data[i_plot][2])],
        text=[np.round(np.mean(q_data[i_plot][2]),2)],
        textposition="auto",
        textfont_color = "white",
        marker_color=px.colors.qualitative.T10[2],            
        error_y=dict(
            type='data',
            symmetric=True,
            thickness=2,
            width=7,
            array=[np.std(q_data[i_plot][2])]),
        )
    )
    
    

fig.update_layout(
    yaxis = dict(range=[0, 80]),
    showlegend = True
    )

fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')

fig.show() 

DTW Groups

In [17]:

offset = 25
max_size = 18

fig = go.Figure()

#Diagonal
fig.add_trace(
            go.Scatter(
                mode='lines',
                x=[0, max_size],
                y=[0, max_size],
                showlegend=False,
                marker=dict(
                    color='gray',
                )
            )
        )

#Plot Datos
for index, row in df.iterrows():
    if(row['grupo'] == 'sano'):
        color_marker = 'red'
    elif(row['grupo'] == 'leve'):
        color_marker = 'green'
    elif(row['grupo'] == 'moderado'):
        color_marker = 'blue'
    fig.add_trace(
            go.Scatter(
                mode='markers',
                x=row.dataframe['mean_dtw_org_error'],
                y=row.dataframe['mean_dtw_error'],
                text=row.dataframe['user'],
                name=row['grupo'],
                marker=dict(
                    color=color_marker,
                )
            )
        )

#Update Layaout
fig.update_layout(
    title="Gráfico DTW",
    xaxis_title="mean DTW distance to True Gesture",
    yaxis_title="mean DTW distance to Predicted Gesture",
    legend_title="Grupos"
    )

fig.show()

Predicción

In [53]:
group = []
dtw = []
pred = []    
for i_group in [1, 0, 2]:    
    df_tmp = df.dataframe[i_group]
    
    group.append(df.grupo[i_group])    
    dtw.append( df_tmp.mean_dtw_error.to_list() )
    pred.append( df_tmp.mean_porc_pred.to_list() )

fig = make_subplots(rows=1, cols=2, subplot_titles=("DTW", "% Predicción"))
for i_plot in range(3):
    
    fig.add_trace(go.Bar(name = group[i_plot][i_plot],
        x=[1+i_plot-0.19],
        y=[np.mean(dtw[i_plot][i_plot])],
        text=[np.round(np.mean(dtw[i_plot]),2)],
        textposition="auto",
        textfont_color = "white",
        marker_color=px.colors.qualitative.T10[i_plot],            
        error_y=dict(
            type='data',
            symmetric=True,
            thickness=2,
            width=7,
            array=[np.std(dtw[i_plot])])
        ),col=1,row=1
    )
    
    fig.add_trace(go.Bar(name = group[i_plot][i_plot],
        x=[1+i_plot+0.19],
        y=[np.mean(pred[i_plot][i_plot])],
        text=[np.round(np.mean(pred[i_plot]),2)],
        textposition="auto",
        textfont_color = "white",
        marker_color=px.colors.qualitative.T10[i_plot],            
        error_y=dict(
            type='data',
            symmetric=True,
            thickness=2,
            width=7,
            array=[np.std(pred[i_plot])])
        ),col=2,row=1
    )

fig.update_layout(
yaxis2 = dict(range=[0, 100]),
showlegend = True
)

fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black')

fig.show() 